In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.functions import *

In [0]:
df = spark.read.load('/FileStore/tables/googleplaystore-3.csv', format='csv', header='true',sep=',',escape='"',inferschema='true')

In [0]:
df.count()

Out[3]: 10841

In [0]:
df.show(1)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [0]:
df=df.drop("Size","Content Rating","Last Updated","Android Ver","Current Ver")

In [0]:
df.show(1)

+--------------------+--------------+------+-------+--------+----+-----+------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|      Genres|
+--------------------+--------------+------+-------+--------+----+-----+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|Art & Design|
+--------------------+--------------+------+-------+--------+----+-----+------------+
only showing top 1 row



In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [0]:
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import IntegerType

df = df.withColumn("Reviews", col("Reviews").cast('integer')) \
    .withColumn("Installs", regexp_replace(col("Installs"), "[^0-9]", "")) \
    .withColumn("Price", regexp_replace(col("Price"), "[$]", "")) \
    .withColumn("Price", col("Price").cast('integer'))

In [0]:
df.show(5)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 5 rows



In [0]:
df.createOrReplaceTempView("apps")

In [0]:
%sql
SELECT * FROM apps
limit 5;

App,Category,Rating,Reviews,Installs,Type,Price,Genres
Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,10000,Free,0,Art & Design
Coloring book moana,ART_AND_DESIGN,3.9,967,500000,Free,0,Art & Design;Pretend Play
"U Launcher Lite – FREE Live Cool Themes, Hide Apps",ART_AND_DESIGN,4.7,87510,5000000,Free,0,Art & Design
Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,50000000,Free,0,Art & Design
Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,100000,Free,0,Art & Design;Creativity


In [0]:
%sql
select 
  App,
  sum(Reviews) 
from apps
group by 1 
order by 2 desc
limit 10;

App,sum(Reviews)
Instagram,266241989
WhatsApp Messenger,207348304
Clash of Clans,179558781
Messenger – Text and Video Chat for Free,169932272
Subway Surfers,166331958
Candy Crush Saga,156993136
Facebook,156286514
8 Ball Pool,99386198
Clash Royale,92530298
Snapchat,68045010


In [0]:
%sql
select 
  App,
  Type,
  sum(Installs) 
from apps
group by 1,2
order by 3 desc
limit 10;

App,Type,sum(Installs)
Subway Surfers,Free,6.0E9
Instagram,Free,4.0E9
Google Drive,Free,4.0E9
Hangouts,Free,4.0E9
Google Photos,Free,4.0E9
Google News,Free,4.0E9
Candy Crush Saga,Free,3.5E9
WhatsApp Messenger,Free,3.0E9
Gmail,Free,3.0E9
Temple Run 2,Free,3.0E9


In [0]:
%sql
select 
   Category,
   sum(Installs)
from apps
group by 1
order by 2 desc
limit 10;

Category,sum(Installs)
GAME,3.5086024415E10
COMMUNICATION,3.2647276251E10
PRODUCTIVITY,1.4176091369E10
SOCIAL,1.4069867902E10
TOOLS,1.1452771915E10
FAMILY,1.0258263505E10
PHOTOGRAPHY,1.0088247655E10
NEWS_AND_MAGAZINES,7.49631776E9
TRAVEL_AND_LOCAL,6.868887146E9
VIDEO_PLAYERS,6.22200272E9


In [0]:
%sql
select
  App,
  sum(Price)
from apps
where Type='Paid'
group by 1
order by 2 desc
limit 10;

App,sum(Price)
I'm Rich - Trump Edition,400
most expensive app (H),399
I am rich(premium),399
I Am Rich Premium,399
I am Rich Plus,399
I'm Rich/Eu sou Rico/أنا غني/我很有錢,399
I Am Rich Pro,399
I AM RICH PRO PLUS,399
I am Rich,399
I am Rich!,399
